This tutorial demonstrates data processing steps specifically as applied to scanning electron diffraction (SED) datasets acquired at the Electron Physical Sciences Imaging Centre (ePSIC) using the Merlin-Medipix detector system installed on E02. Helfpul code snippets are provided to deal with the file structures and formats associated with this system.

# Change Log

25/03/18 Duncan Johnstone - Developed for ePSIC HyperSpy Workshop

# Requirements

pyXem 0.5

HyperSpy 1.3

PyMatGen

# Materials & Methods

Data used in this useage example was acquired from a calibration standard Au cross-grating (Agar Scientific) 

# Contents

1. <a href='#loa'> Loading & Data Conversion</a>
2. <a href='#vdf'> Inspection & Virtual Diffraction Imaging</a>
3. <a href='#vec'> Peak Finding & Vector Analysis</a>

Import pyXem and other required libraries

In [ ]:
%matplotlib tk
import pyxem as pxm
import numpy as np
import os

# <a id='loa'></a> 1. Loading and Data Conversion

Create a list of file names for iteration by walking through timestamped directories

In [ ]:
filenames = []
for root, dirs, files in os.walk("./demo_mib_data/"):
    for file in files:
        if file.endswith(".hdr"):
             filenames.append(os.path.join(root, file))

In [ ]:
filenames

Define corresponding lists of calibration values based on experimental conditions

In [ ]:
#Both datasets acquired with same camera length, this value is A-1 / pixel
recip_cals = [0.00889, 0.00889]
#Scans performed at different magnifications, this value is nm/pixel
real_cals = [5.0, 4.0]

Select one example of each of the above parameters

In [ ]:
fname = filenames[0]
tstamp = os.path.dirname(fname)[-6:]
recip_cal = recip_cals[0]
real_cal = real_cals[0]

Load dataset, crop direct beam region and perform alignment

In [ ]:
dp = pxm.load_mib(fname, scan_size=256)
#crop center
roi = pxm.roi.RectangularROI(left=103, top=103, right=153, bottom=153)
dpc = roi(dp, axes=dp.axes_manager.signal_axes)
#perform alignment
centers = dpc.get_direct_beam_position(sigma=3)
shifts = centers.data - np.array((25,25))
shifts = shifts.reshape(65025, 2)
dp.align2D(shifts=shifts, crop=False, fill_value=0)

Set calibration values

In [ ]:
dp.set_diffraction_calibration(recip_cal)
dp.set_scan_calibration(real_cal)

Save data, representative summed diffraction, and virtual images

In [ ]:
dp.save('./epsic_eg_' + tstamp)
#summed diffraction
dpeg = dp.sum((0,1))
dpeg.change_dtype('float32')
dpeg.save('./epsic_eg_dpeg_' + tstamp + '.tif')
#vbf image
circ = pxm.roi.CircleROI(cx=0, cy=0, r=10*recip_cal)
vbf = dp.get_virtual_image(circ)
vbf = vbf.as_signal2D((0,1))
vbf.change_dtype('float32')
vbf.save('./epsic_eg_vbf_' + tstamp + '.tif')
#vdf image
ann = pxm.roi.CircleROI(cx=0, cy=0, r=100*recip_cal, r_inner=50*recip_cal)
vdf = dp.get_virtual_image(ann)
vdf = vdf.as_signal2D((0,1))
vdf.change_dtype('float32')
vdf.save('./epsic_eg_vdf_' + tstamp + '.tif')

The steps above can be combined into a loop, as follows.

In [ ]:
i=0
for fname in filenames:
    tstamp = os.path.dirname(fname)[-6:]
    #load data
    dp = pxm.load_mib(fname, scan_size=256)
    #crop center
    roi = pxm.roi.RectangularROI(left=103, top=103, right=153, bottom=153)
    dpc = roi(dp, axes=dp.axes_manager.signal_axes)
    #perform alignment
    centers = dpc.get_direct_beam_position(sigma=3)
    shifts = centers.data - np.array((25,25))
    shifts = shifts.reshape(65025, 2)
    dp.align2D(shifts=shifts, crop=False, fill_value=0)
    #set calibrations
    dp.set_diffraction_calibration(recip_cals[i])
    dp.set_scan_calibration(real_cals[i])
    #save data
    dp.save('./epsic_eg_' + tstamp)
    #summed diffraction
    dpeg = dp.sum((0,1))
    dpeg.change_dtype('float32')
    dpeg.save('./epsic_eg_dpeg_' + tstamp + '.tif')
    #vbf image
    circ = pxm.roi.CircleROI(cx=0, cy=0, r=10*recip_cals[i])
    vbf = dp.get_virtual_image(circ)
    vbf = vbf.as_signal2D((0,1))
    vbf.change_dtype('float32')
    vbf.save('./epsic_eg_vbf_' + tstamp + '.tif')
    #vdf image
    ann = pxm.roi.CircleROI(cx=0, cy=0, r=100*recip_cals[i], r_inner=50*recip_cals[i])
    vdf = dp.get_virtual_image(ann)
    vdf = vdf.as_signal2D((0,1))
    vdf.change_dtype('float32')
    vdf.save('./epsic_eg_vdf_' + tstamp + '.tif')
    i=i+1

# 2. <a href='#vdf'></a> Inspection & Virtual Diffraction Imaging

Load the converted data and reset calibrations

In [ ]:
dp = pxm.load('./epsic_eg_111859.hspy')
dp = pxm.ElectronDiffraction(dp)
dp.set_diffraction_calibration(recip_cal)
dp.set_scan_calibration(real_cal)

Inspect the data via virtual imaging

In [ ]:
roi = pxm.roi.CircleROI(cx=0.,cy=0, r_inner=0, r=0.07)
dp.plot_interactive_virtual_image(roi=roi)

# 3. <a href='#vec'> </a> Peak Finding & Vector Analysis

Choose a region to crop to keep things faster

In [1]:
reg = pxm.roi.RectangularROI(left=50, top=750, right=290, bottom=990)
dp.plot()
reg.add_widget(dp)

NameError: name 'pxm' is not defined

Crop the region

In [ ]:
dpc = reg(dp)

Find the peaks

In [ ]:
peaks = dpb.find_peaks(method='difference_of_gaussians',
                        min_sigma=1.,
                        max_sigma=6.,
                        sigma_ratio=1.6,
                        threshold=0.04,
                        overlap=0.99)

Inspect the lengths of the found peaks

In [ ]:
bins = np.arange(0, 1.5, recip_cal)
ghist = peaks.get_magnitude_histogram(bins=bins)
ghist.plot()